In [ ]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from sklearn.preprocessing import StandardScaler

# For reproducible results
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# Paths to the .npy files
categories_path = '/Users/rishabhtiwari/Desktop/training_dataset/categories.npy'
valid_periods_path = '/Users/rishabhtiwari/Desktop/training_dataset/valid_periods.npy'
training_data_path = '/Users/rishabhtiwari/Desktop/training_dataset/training_data.npy'

# Load the data
categories = np.load(categories_path)
valid_periods = np.load(valid_periods_path)
training_data = np.load(training_data_path)

In [ ]:
# Constants for sequence and forecast lengths
seq_length = 128     # predictions based on previous seq_length data entries
forecast_length = 9  # predicting forecast_length time steps into the future

# Data preprocessing
scaler = StandardScaler()

In [ ]:
# Data preprocessing
scaler = StandardScaler()

def preprocess_data(data, valid_periods, scaler):
    preprocessed_data = []
    for i, row in enumerate(data):
        valid_data = row[valid_periods[i][0]:valid_periods[i][1]]
        if valid_data.size > 0:
            scaled_data = scaler.fit_transform(valid_data.reshape(-1, 1)).flatten()
            preprocessed_data.append(scaled_data)
    return preprocessed_data

In [ ]:
# Convert the time series data into sequences
def to_sequences(data):
    X, y = [], []
    for ts in data:
        for i in range(len(ts) - seq_length - forecast_length + 1):
            X.append(ts[i:(i + seq_length)])
            y.append(ts[(i + seq_length):(i + seq_length + forecast_length)])
    return np.array(X), np.array(y)

In [ ]:
# Preprocess the data and convert to sequences
data = preprocess_data(training_data, valid_periods, scaler)
X, y = to_sequences(data)

In [ ]:
# Shuffle and split the data into training and validation sets
val_size = int(len(X) * 0.2)
indices = np.arange(len(X))
np.random.shuffle(indices)
X_train, y_train = X[:-val_size], y[:-val_size]
X_val, y_val = X[-val_size:], y[-val_size:]

In [ ]:
# Reshape data for LSTM input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], 1))

In [ ]:
# Model building
def build_model(input_shape, lstm_units, dropout_rate, forecast_length):
    input_layer = tfkl.Input(shape=input_shape)
    x = tfkl.Bidirectional(tfkl.LSTM(lstm_units, return_sequences=True))(input_layer)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    x = tfkl.LSTM(lstm_units // 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    output_layer = tfkl.Dense(forecast_length)(x)

    model = tfk.Model(inputs=input_layer, outputs=output_layer)
    return model

input_shape = (X_train.shape[1], X_train.shape[2])
dropout_rate = 0.2
lstm_units = 128

In [ ]:
# Build the model
model = build_model(input_shape, lstm_units, dropout_rate, forecast_length)

In [ ]:
# Compile the model
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
# Train the model
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(
    X_train,
    y_train,
    batch_size=128,
    epochs=1000,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=1
)